In [41]:
print('This notebook was prepared using:')

import os
import sys
print('python version %s' % sys.version)

import numpy as np
print('numpy version %s' % np.__version__)

import pandas as pd
print('pandas version %s' % pd.__version__)

import math
from math import isnan

import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.image as mpimg
print('matplotlib version %s' % matplotlib.__version__)

import re
import sys
import argparse
import scipy as sp
try:
    import itertools.izip as zip
except ImportError:
    import itertools
import random

from scipy import integrate
import scipy.interpolate as sp_interpolate
import statistics

from dataclasses import dataclass
import time as time_module

# GitHub
# HIV_DIR = 'data/HIV200'
HIV_DIR = 'data/HIV'
SIM_DIR = 'data/simulation'
FIG_DIR = 'figures'
MPL_DIR = 'src'

This notebook was prepared using:
python version 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:26:08) [Clang 14.0.6 ]
numpy version 1.24.2
pandas version 1.5.3
matplotlib version 3.7.1


1. all epitopes just had one or two escape mutations (700010077-5,700010058-5,700010040-5,706010164-5,704010042-5)
2. all epitopes have more than two escape mutations (700010077-3,700010058-3,700010607-3,705010198-3,705010185-5,703010159-3) 
3. combination of the above (700010470-3 ,700010470-5 ,700010040-3,706010164-3, 705010162-3, 705010162-5, 704010042-3, 703010256-3, 703010256-5, 703010131-3, 703010131-5)
4. no mutation related to escape (700010607-5,705010198-5,705010185-3,703010159-5)

tag|sequence<br>length|escape<br>group|variants|max<br>time|time<br>step|gamma<br>s|raw<br>time|special<br>sites
:----:|:----:|:---:|:-----:|:----|:----|:----:|:----:|:----:
700010040-3|303|3|314|552|20|0.018|[0, 16, 45, 111, 181, 283, 412, 552]|[245, 246]
700010040-5|146|0|146|552|5 |0.018|[0, 16, 45, 111, 181, 283, 412, 552]|[31, 32, 39, 104]
700010058-3|90 |1|91 |85 |1 |0.118|[0, 8, 45, 85]|[]
700010058-5|96 |0|96 |350|1 |0.029|[0, 8, 45, 85, 154, 239, 252, 350]|[13, 16, 17]
700010077-3|203|6|221|159|5 |0.063|[0, 14, 32, 102, 159]|[]
700010077-5|48 |0|49 |159|1 |0.063|[0, 14, 32, 159]|[1, 4, 5, 47]
700010470-3|367|3|405|420|20|0.024|[0, 13, 41, 69, 174, 420]|[40, 334]
700010470-5|193|3|198|454|5 |0.022|[0, 13, 41, 69, 174, 420, 454]|[8, 9, 124]
700010607-3|239|1|252|21 |1 |0.476|[0, 9, 14, 21]|[]
700010607-5|78 |0|78 |21 |1 |     |[0, 9, 14, 21]|no mutation related to epitope	
703010131-3|744|1|807|333|20|0.030|[0, 21, 28, 34, 63, 91, 175, 273, 333]|[0, 45, 46, 47, 76, 189, 560, 561]
703010131-5|261|1|265|333|5 |0.030|[0, 21, 28, 34, 63, 91, 175, 273, 333]|[259]
703010159-3|477|2|517|302|20|0.033|[0, 8, 12, 22, 29, 56, 85, 302]|[]
703010159-5|216|0|224|302|5 |     |[0, 8, 12, 22, 29, 56, 85, 302]|no mutation related to epitope	
703010256-3|463|4|510|684|20|0.015|[0, 28, 63, 172, 426, 684]|[45, 46, 316, 317]
703010256-5|402|1|410|684|20|0.015|[0, 28, 63, 172, 426, 684]|[42, 208]
704010042-3|875|2|1074|676|20|0.015|[0, 21, 60, 172, 424, 676]|[95, 96, 765, 766]
704010042-5|266|0|272|676|5|0.015|[0, 21, 60, 172, 424, 676]|[29, 30, 125]
705010162-3|508|2|562|438|20|0.023|[0, 21, 77, 179, 438]|[33, 34, 327, 328]
705010162-5|254|3|259|438|5 |0.023|[0, 21, 77, 179, 438]|[155]
705010185-3|292|0|303|416|5 |     |[0, 25, 67, 180, 416]|no mutation related to epitope	
705010185-5|85 |1|89 |67 |1 |0.149|[0, 25, 67]|[]
705010198-3|204|1|223|60 |1 |0.167|[0, 11, 60]|[]
705010198-5|72 |0|73 |60 |1 |     |[0, 11, 60]|no mutation related to epitope	
706010164-3|485|3|532|434|20|0.023|[0, 14, 28, 70, 183, 434]|[463]
706010164-5|204|0|205|434|5 |0.023|[0, 14, 28, 70, 183, 434]|[44, 50, 51, 170]

Write sh file for HIV data

In [90]:
infer_tags = ['700010040-3','700010040-5','700010058-3','700010058-5','700010077-3','700010077-5',
              '700010470-3','700010470-5','700010607-3','703010131-3','703010131-5','703010159-3',
              '703010256-3','703010256-5','704010042-5','705010162-3','705010162-5',
              '705010185-5','705010198-3','706010164-3','706010164-5'] #'704010042-3'

f = open("%s/HIV_run.sh"%MPL_DIR,'w')
f.write('#!/bin/bash\n')

# f.write('tags=(')
# for tag in infer_tags:
#     f.write('"%s" '%tag)
# f.write(')\n')
# f.write('for tag in "${tags[@]}"\n')
# f.write('do\n')
# f.write('   python ../inference_HIV.py -tag %s --raw & \n'%(tag))
# f.write('done\n')
# f.write('wait\n')
# f.write('echo "All scripts have been attempted."')
# f.close()  

for i in range(len(infer_tags)):
    tag = infer_tags[i]
    f.write('\tpython ../inference_HIV.py -tag %s --raw'%(tag))
    f.write(' || echo "CH%s failed, continuing with next script..." \n'%tag[-5:])
f.write('echo "All scripts have been attempted."')
f.close()  

Write sh file for HIV data with shorter time


tag|sequence<br>length|escape<br>group|variants|max<br>time|time<br>step|gamma<br>s|raw<br>time|special<br>sites
:----:|:----:|:---:|:-----:|:----|:----|:----:|:----:|:----:
700010470-3|282|3|301|174|5|0.057|[0, 13, 41, 69, 174]|[32]|
700010470-5|139|1|141|174|5|0.057|[0, 13, 41, 69, 174]|[8, 17, 57, 59, 90]|
703010159-3|312|2|322|85|1|0.118|[0, 8, 12, 22, 29, 56, 85]|[]|
703010256-3|175|3|180|172|5|0.058|[0, 28, 63, 172]|[20, 105, 159, 160]|
703010256-5|236|1|238|172|5|0.058|[0, 28, 63, 172]|[]|
704010042-3|499|1|585|172|20|0.058|[0, 21, 60, 172]|[20, 21, 51, 52, 426]|
704010042-5|80|0|80|172|1|0.058|[0, 21, 60, 172]|[9, 10]|
705010162-3|394|2|404|179|20|0.056|[0, 21, 77, 179]|[18, 19, 224]|
705010162-5|135|2|135|179|5|0.056|[0, 21, 77, 179]|[2, 3]|
706010164-3|275|3|280|183|5|0.055|[0, 14, 28, 70, 183]|[269]|
706010164-5|109|0|110|183|5|0.055|[0, 14, 28, 70, 183]|[27, 93]|


In [ ]:
tags = ['700010470-3','700010470-5','703010159-3','703010256-3','703010256-5','704010042-3','704010042-5',
        '705010162-3','705010162-5','706010164-3','706010164-5']

f = open("%s/shorter.sh"%MPL_DIR,'w')
for tag in tags:
    f.write('python ../inference_HIV_shorter.py -tag %s --raw\n'%(tag))
f.close()          


In [10]:
NUC = ['-', 'A', 'C', 'G', 'T']

tags = ['700010470-3','700010470-5','703010159-3','703010256-3','703010256-5','704010042-3','704010042-5',
        '705010162-3','705010162-5','706010164-3','706010164-5']

for i in range(len(tags)):
    tag = tags[i]
    result = AnalyzeData(tag)
    print('%s|%d|%d|%d|%d|%d|%s|'%(tag,result.seq_length,len(result.escape_group),result.variants,\
                                   result.uniq_t[-1],result.time_step,result.gamma),end='')
    print([int(i) for i in result.uniq_t],end='')
    print('|',end='')
    print([int(i) for i in result.special_sites],end='')
    print('|')

700010470-3|282|3|301|174|5|0.057|[0, 13, 41, 69, 174]|[32]|
700010470-5|139|1|141|174|5|0.057|[0, 13, 41, 69, 174]|[8, 17, 57, 59, 90]|
703010159-3|312|2|322|85|1|0.118|[0, 8, 12, 22, 29, 56, 85]|[]|
703010256-3|175|3|180|172|5|0.058|[0, 28, 63, 172]|[20, 105, 159, 160]|
703010256-5|236|1|238|172|5|0.058|[0, 28, 63, 172]|[]|
704010042-3|499|1|585|172|20|0.058|[0, 21, 60, 172]|[20, 21, 51, 52, 426]|
704010042-5|80|0|80|172|1|0.058|[0, 21, 60, 172]|[9, 10]|
705010162-3|394|2|404|179|20|0.056|[0, 21, 77, 179]|[18, 19, 224]|
705010162-5|135|2|135|179|5|0.056|[0, 21, 77, 179]|[2, 3]|
706010164-3|275|3|280|183|5|0.055|[0, 14, 28, 70, 183]|[269]|
706010164-5|109|0|110|183|5|0.055|[0, 14, 28, 70, 183]|[27, 93]|


In [3]:
from dataclasses import dataclass

@dataclass
class Result:
    variants: 0
    seq_length: 0
    special_sites: []
    uniq_t:[]
    time_step:0
    gamma:0
    escape_group:[]
    escape_TF:[]
    trait_dis:[]
    IntTime:[]
        
def AnalyzeData(tag,gamma1):
    df_info = pd.read_csv('%s/analysis/%s-analyze.csv' %(HIV_DIR,tag), comment='#', memory_map=True)
    seq     = np.loadtxt('%s/sequence/%s-poly-seq2state.dat'%(HIV_DIR,tag))

    """get raw time points"""
    times = []
    for i in range(len(seq)):
        times.append(seq[i][0])
    uniq_t = np.unique(times)

    """get variants number and sequence length"""
    df_poly  = df_info[df_info['nucleotide']!=df_info['TF']]
    variants = len(df_poly)
    seq_length = df_info.iloc[-1].polymorphic_index + 1

    """get special sites and escape sites"""
    # get all epitopes for one tag
    df_rows = df_info[df_info['epitope'].notna()]
    unique_epitopes = df_rows['epitope'].unique()

    min_n = 2 # the least escape sites a trait group should have (more than min_n)
    special_sites = [] # special site considered as time-varying site but not escape site
    escape_group  = [] # escape group (each group should have more than 2 escape sites)
    escape_TF     = [] # corresponding wild type nucleotide
    for epi in unique_epitopes:
        df_e = df_rows[(df_rows['epitope'] == epi) & (df_rows['escape'] == True)] # find all escape mutation for one epitope
        unique_sites = df_e['polymorphic_index'].unique()

        if len(unique_sites) <= min_n:
            special_sites.append(unique_sites)
        else:
            escape_group.append(list(unique_sites))
            tf_values = []
            for site in unique_sites:
                tf_value = df_e[df_e['polymorphic_index'] == site]['TF'].values
                tf_values.append(NUC.index(tf_value[0]))
            escape_TF.append(tf_values)

    special_sites = [item for sublist in special_sites for item in sublist]

    """trait distance"""
    trait_dis = []
    if len(escape_group) > 0:
        df_sequence = pd.read_csv('%s/processed/%s-index.csv' %(HIV_DIR,tag), comment='#', memory_map=True,usecols=['alignment','polymorphic'])
        for i in range(len(escape_group)):
            i_dis = []
            for j in range(len(escape_group[i])-1):
                index0 = df_sequence[df_sequence['polymorphic']==escape_group[i][j]].iloc[0].alignment
                index1 = df_sequence[df_sequence['polymorphic']==escape_group[i][j+1]].iloc[0].alignment
                i_dis.append(int(index1-index0))
            trait_dis.append(i_dis)

    """find proper time step"""
    if uniq_t[-1] < 100:
        time_step = 1
    else:
        if seq_length > 300:
            time_step = 20
        elif seq_length < 100:
            time_step = 1
        else:
            time_step = 5

    """find proper gamma value"""
    # interpolation time according to the time step get above, all the inserted time points are integer
    times = [0]
    for t in range(1,len(uniq_t)):
        tp_i   = round((uniq_t[t]-uniq_t[t-1])/time_step) # number of insertion points
        if tp_i > 0:
            ts_i   = round((uniq_t[t]-uniq_t[t-1])/tp_i) # modified time step
            for i in range(tp_i):
                time_i = uniq_t[t-1] + (i+1) * ts_i
                time_s = uniq_t[t]
                # if the inserted time is close to the bounadry time point, throw out the inserted point
                if abs(time_i-time_s) <= ts_i/2:
                    times.append(int(time_s))
                else:
                    times.append(int(time_i))
        else:
            times.append(uniq_t[t])
    IntTime = list(times)

    # regularize value gamma1, restricting the magnitude of the selection coefficients
    gamma = round(gamma1/uniq_t[-1],3) # constant MPL gamma value / max time

    return Result(variants, seq_length, special_sites, uniq_t, time_step, gamma, escape_group, escape_TF, trait_dis, IntTime)

In [69]:
# loading data from dat file
def getSequence(history,escape_TF,escape_group):
    sVec      = []
    nVec      = []
    eVec      = []

    temp_sVec   = []
    temp_nVec   = []
    temp_eVec   = []

    times       = []
    time        = 0
    times.append(time)

    ne          = len(escape_group)

    for t in range(len(history)):
        if history[t][0] != time:
            time = history[t][0]
            times.append(int(time))
            sVec.append(temp_sVec)
            nVec.append(temp_nVec)
            eVec.append(temp_eVec)
            temp_sVec   = []
            temp_nVec   = []
            temp_eVec   = []

        temp_nVec.append(history[t][1])
        temp_sVec.append(history[t][2:])

        if ne > 0: # the patient contains escape group
            temp_escape = np.zeros(ne, dtype=int)
            for n in range(ne):
                for nn in range(len(escape_group[n])):
                    index = escape_group[n][nn] + 2
                    if history[t][index] != escape_TF[n][nn]:
                        temp_escape[n] = 1
                        break
            temp_eVec.append(temp_escape)

        if t == len(history)-1:
            sVec.append(temp_sVec)
            nVec.append(temp_nVec)
            eVec.append(temp_eVec)

    return sVec,nVec,eVec

# get muVec
def getMutantS(sVec):
    # use muVec matrix to record the index of time-varying sites
    muVec = -np.ones((seq_length, q)) # default value is -1, positive number means the index
    x_length  = 0

    for i in range(seq_length):            
        # find all possible alleles in site i
        alleles     = [int(sVec[t][k][i]) for t in range(len(sVec)) for k in range(len(sVec[t]))]
        allele_uniq = np.unique(alleles)
        for allele in allele_uniq:
            muVec[i][int(allele)] = x_length
            x_length += 1

    return x_length,muVec

# calculate single and pair allele frequency (multiple case)
def get_allele_frequency(sVec,nVec,eVec,muVec):

    x  = np.zeros((len(nVec),x_length))           # single allele frequency
    xx = np.zeros((len(nVec),x_length,x_length))  # pair allele frequency
    for t in range(len(nVec)):
        pop_size_t = np.sum([nVec[t]])
        for k in range(len(nVec[t])):
            # individual locus part
            for i in range(seq_length):
                qq = int(sVec[t][k][i])
                aa = int(muVec[i][qq])
                if aa != -1: # if aa = -1, it means the allele does not exist
                    x[t,aa] += nVec[t][k]
                    for j in range(int(i+1), seq_length):
                        qq = int(sVec[t][k][j])
                        bb = int(muVec[j][qq])
                        if bb != -1:
                            xx[t,aa,bb] += nVec[t][k]
                            xx[t,bb,aa] += nVec[t][k]
            # escape part
            for n in range(ne):
                aa = int(x_length-ne+n)
                x[t,aa] += eVec[t][k][n] * nVec[t][k]
                for m in range(int(n+1), ne):
                    bb = int(x_length-ne+m)
                    xx[t,aa,bb] += eVec[t][k][n] * eVec[t][k][m] * nVec[t][k]
                    xx[t,bb,aa] += eVec[t][k][n] * eVec[t][k][m] * nVec[t][k]
                for j in range(seq_length):
                    qq = int(sVec[t][k][j])
                    bb = int(muVec[j][qq])
                    if bb != -1:
                        xx[t,bb,aa] += eVec[t][k][n] * nVec[t][k]
                        xx[t,aa,bb] += eVec[t][k][n] * nVec[t][k]
        x[t,:]    = x[t,:]/pop_size_t
        xx[t,:,:] = xx[t,:,:]/pop_size_t
    return x,xx

# calculate escape frequency
def get_escape_fre_term(sVec,nVec):
    ex  = np.zeros((len(nVec),ne,seq_length,q))
    for t in range(len(nVec)):
        pop_size_t = np.sum([nVec[t]])
        for k in range(len(sVec[t])):
            for n in range(ne):
                n_mutations = 0
                for nn in escape_group[n]:
                    index = escape_group[n].index(nn)
                    WT = escape_TF[n][index]
                    if sVec[t][k][nn] != WT:
                        n_mutations += 1
                        site = nn
                if n_mutations == 1:
                    qq = int(sVec[t][k][site])
                    ex[t,n,site,qq] += nVec[t][k]
        ex[t,:,:,:] = ex[t,:,:,:] / pop_size_t
    return ex

# calculate frequencies for recombination part (binary case)  
def get_p_k(sVec,nVec,seq_length,escape_group,escape_TF):
    p_mut_k   = np.zeros((len(nVec),seq_length,3)) # 0: time, 1: all k point, 2: p_k, p_k-, p_k+
    p_wt      = np.zeros((len(nVec),len(escape_group))) # 0: time, 1: escape group

    for t in range(len(nVec)):
        pop_size_t = np.sum([nVec[t]])

        for n in range(len(escape_group)):
            escape_group_n = escape_group[n]
            sWT_n     = [int(i) for i in escape_TF[n]]

            for k in range(len(sVec[t])): # different sequences at time t
                sVec_n = [int(sVec[t][k][i]) for i in escape_group_n]

                # no mutation within the trait group
                if sWT_n == sVec_n:
                    p_wt[t][n] += nVec[t][k]

                for nn in range(len(escape_group_n)-1):
                    k_bp = nn + 1

                    # containing mutation before and after break point k,p_k
                    if sWT_n[:k_bp] != sVec_n[:k_bp] and sWT_n[k_bp:] != sVec_n[k_bp:]:
                        p_mut_k[t][escape_group_n[0]+nn][0] += nVec[t][k]

                    # MT before break point k and WT after break point k,p_k-
                    if sWT_n[:k_bp] != sVec_n[:k_bp] and sWT_n[k_bp:] == sVec_n[k_bp:]:
                        p_mut_k[t][escape_group_n[0]+nn][1] += nVec[t][k]

                    # WT before break point k and MT after break point k,p_k+
                    if sWT_n[:k_bp] == sVec_n[:k_bp] and sWT_n[k_bp:] != sVec_n[k_bp:]:
                        p_mut_k[t][escape_group_n[0]+nn][2] += nVec[t][k]

        p_wt[t]    = p_wt[t] / pop_size_t
        p_mut_k[t] = p_mut_k[t] / pop_size_t

    return p_wt,p_mut_k

# calculate mutation flux term 
def get_mutation_flux(x,ex,muVec):
    flux = np.zeros((len(x),x_length))
    for t in range(len(x)):
        for i in range(seq_length):
            for a in range(q):
                aa = int(muVec[i][a])
                if aa != -1:
                    for b in range(q):
                        bb = int(muVec[i][b])
                        if b != a:
                            if bb != -1:
                                flux[t,aa] +=  muMatrix[b][a] * x[t,bb] - muMatrix[a][b] * x[t,aa]
                            else:
                                flux[t,aa] += -muMatrix[a][b] * x[t,aa]
        for n in range(ne):
            for nn in range(len(escape_group[n])):
                for a in range(q):
                    WT = escape_TF[n][nn]
                    index = escape_group[n][nn]
                    if a != WT:
                        flux[t, x_length-ne+n] += muMatrix[WT][a] * (1 - x[t,x_length-ne+n]) - muMatrix[a][WT] * ex[t,n,index,a]
    return flux

# calculate recombination flux term
def get_recombination_flux(x,p_wt,p_mut_k,trait_dis):
    flux = np.zeros((len(x),x_length))
    for n in range(ne):
        for t in range(len(x)):
            fluxIn  = 0
            fluxOut = 0

            for nn in range(len(escape_group[n])-1):
                k_index = escape_group[n][0]+nn
                fluxIn  += trait_dis[n][nn] * p_wt[t][n]*p_mut_k[t][k_index][0]
                fluxOut += trait_dis[n][nn] * p_mut_k[t][k_index][1]*p_mut_k[t][k_index][2]

            flux[t,x_length-ne+n] = r_rate * (fluxIn - fluxOut)

    return flux

# diffusion matrix C
def diffusion_matrix_at_t(x,xx):
    C = np.zeros([len(x),x_length,x_length])
    for t in range(len(x)):
        for i in range(x_length):
            C[t,i,i] = x[t,i] - x[t,i] * x[t,i]
            for j in range(int(i+1) ,x_length):
                C[t,i,j] = xx[t,i,j] - x[t,i] * x[t,j]
                C[t,j,i] = xx[t,i,j] - x[t,i] * x[t,j]
    return C

# calculate the difference between the frequency at time t and time t-1
def cal_delta_x(single_freq,times):
    delta_x = np.zeros((len(single_freq),x_length))   # difference between the frequency at time t and time t-1s
#     calculate by np.gradient function
#         for ii in range(x_length):
#             delta_x[:,ii] = np.gradient(single_freq.T[ii],times)
#     calculate manually
    for t in range(len(single_freq)-1):
        delta_x[t] = (single_freq[t+1] - single_freq[t])/(times[t+1]-times[t])
    delta_x[-1] = delta_x[-2]
    return delta_x

# Interpolation function definition
# Interpolates the input arrays so that they will have the same number of generations as the original population.
interpolation = lambda a,b: sp_interpolate.interp1d(a,b,kind='linear')

def interpolator_x(single_freq, double_freq, current_times, result_times):
    single_freq_temp = np.zeros((len(result_times),x_length))
    double_freq_temp = np.zeros((len(result_times),x_length,x_length))

    for i in range(x_length):
        single_freq_temp[:,i] = interpolation(current_times, single_freq[:,i])(result_times)
        for j in range(x_length):
            double_freq_temp[:,i,j] = interpolation(current_times, double_freq[:,i,j])(result_times)

    return single_freq_temp, double_freq_temp

def interpolator_ex(escape_freq, current_times, result_times):
    if ne != 0:
        escape_freq_temp  = np.zeros((len(result_times),ne,seq_length,q))
    else: # if there is no escape group, return 0
        escape_freq_temp  = []

    for n in range(ne):
        for i in range(seq_length):
            for a in range(q):
                escape_freq_temp[:,n,i,a] = interpolation(current_times, escape_freq[:,n,i,a])(result_times)

    return escape_freq_temp

def interpolator_p(p_wt, p_mut_k, current_times, result_times):
    wt_temp    = np.zeros((len(result_times),ne))
    mut_k_temp = np.zeros((len(result_times),seq_length,3))
    # interpolation for wild type frequency
    for n in range(ne):
        wt_temp[:,n] = interpolation(current_times, p_wt[:,n])(result_times)

    # interpolation for frequency related to recombination part
    for i in range(seq_length):
        for j in range(3):
            mut_k_temp[:,i,j] = interpolation(current_times, p_mut_k[:,i,j])(result_times)

    wt_temp    = wt_temp[:len(result_times)]
    mut_k_temp = mut_k_temp[:len(result_times)]

    return wt_temp, mut_k_temp

In [85]:
tag = '700010058-3'

data     = np.loadtxt("%s/sequence/%s-poly-seq2state.dat" %(HIV_DIR,tag))

# information for escape group
result       = AnalyzeData(tag,10)
escape_group = result.escape_group
escape_TF    = result.escape_TF
trait_dis    = result.trait_dis
seq_length   = result.seq_length
sample_times = result.uniq_t
times        = result.IntTime
ne           = len(escape_group)

## regularization parameter
p_sites      = result.special_sites
time_step    = result.time_step
gamma_1s     = result.gamma

# obtain sequence data and frequencies
sVec,nVec,eVec = getSequence(data,escape_TF,escape_group)
x_length,muVec = getMutantS(sVec)
x_length      += ne

# get all frequencies, 
# x: single allele frequency, xx: pair allele frequency
x,xx         = get_allele_frequency(sVec,nVec,eVec,muVec)

# ex: escape frequency
if ne != 0:
    ex      = get_escape_fre_term(sVec,nVec)
else:
    ex      = 0

# ex: escape frequency, p_wt,p_mut_k: frequency related to recombination part
p_wt,p_mut_k = get_p_k(sVec,nVec,seq_length,escape_group,escape_TF)

#record all input information before interpolation
f = open('%s/rawdata/rawdata_%s.npz'%(HIV_DIR,tag), mode='w+b')
escape_group = np.array(escape_group, dtype=object)
escape_TF    = np.array(escape_TF , dtype=object)
trait_dis    = np.array(trait_dis , dtype=object)
np.savez_compressed(f, muVec=muVec, single_freq=x, double_freq=xx, escape_freq=ex, p_wt_freq=p_wt, p_mut_k_freq=p_mut_k,\
                    special_sites=p_sites, escape_group=escape_group, escape_TF=escape_TF,trait_dis=trait_dis,\
                    seq_length=seq_length, gamma = gamma_1s, time_step=time_step, sample_times=sample_times, times=times)
f.close()

In [86]:
r_rate = 1.4e-5
muMatrix = np.loadtxt("%s/Zanini-extended.dat"%HIV_DIR)

# load processed data from rawdata file

rawdata  = np.load('%s/rawdata/rawdata_%s.npz'%(HIV_DIR,tag), allow_pickle=True)
# information for individual sites
x            = rawdata['single_freq']
xx           = rawdata['double_freq']
ex           = rawdata['escape_freq']
muVec        = rawdata['muVec']
sample_times = rawdata['sample_times']
times        = rawdata['times']
time_step    = rawdata['time_step']
seq_length   = rawdata['seq_length']

# information for escape group
p_wt         = rawdata['p_wt_freq']
p_mut_k      = rawdata['p_mut_k_freq']
p_sites      = rawdata['special_sites']
escape_group = rawdata['escape_group'].tolist()
escape_TF    = rawdata['escape_TF'].tolist()

ne           = len(escape_group)
x_length     = len(x[0])

# regularization value gamma_1 and gamma_2
# individual site: gamma_1s, escape group: gamma_1p
gamma_1s     = rawdata['gamma']
gamma_1p     = gamma_1s/10
gamma1 = np.ones(x_length)*gamma_1s
for n in range(ne):
    gamma1[x_length-ne+n] = gamma_1p

# individual site: gamma_2c, escape group and special site: gamma_2tv
gamma_2c  = 100000
gamma_2tv = 500
gamma2 = np.ones(x_length)*gamma_2c
for n in range(ne):
    gamma2[x_length-ne+n] = gamma_2tv
for p_site in p_sites: # special site - time varying
    for qq in range(len(NUC)):
        index = int (muVec[p_site][qq]) 
        if index != -1:
            gamma2[index] = gamma_2tv

# after interpolation, calculate all the required data
single_freq, double_freq = interpolator_x(x, xx, sample_times, times)
escape_freq              = interpolator_ex(ex, sample_times, times)
p_wt_freq, p_mut_k_freq  = interpolator_p(p_wt, p_mut_k,sample_times, times)

# covariance matrix, flux term and delta_x
covariance_n = diffusion_matrix_at_t(single_freq, double_freq)
covariance   = np.swapaxes(covariance_n, 0, 2)
flux_mu      = get_mutation_flux(single_freq,escape_freq,muVec)         # mutation part
flux_rec     = get_recombination_flux(single_freq,p_wt_freq,p_mut_k_freq,trait_dis) # recombination part
delta_x      = cal_delta_x(single_freq,times)

# extend the time range
TLeft   = int(round(times[-1]*0.5/10)*10)
TRight  = int(round(times[-1]*0.5/10)*10)
etleft  = np.linspace(-TLeft,-10,int(TLeft/10))
etright = np.linspace(times[-1]+10,times[-1]+TRight,int(TRight/10))
ExTimes = np.concatenate((etleft, times, etright))

start_time = time_module.time()

# solve the bounadry condition ODE to infer selections
def fun(a,b):
    """ Function defining the right-hand side of the system of ODE's"""
    b_1                 = b[:x_length,:]   # the actual selection coefficients
    b_2                 = b[x_length:,:]   # the derivatives of the selection coefficients, s'
    result              = np.zeros((2*x_length,len(a))) # The RHS of the system of ODE's
    result[:x_length]   = b_2       # sets the derivatives of the selection coefficients 'b_1', equal to s'
    mat_prod            = np.sum(covariance[:,:,:len(a)] * b_1[:,len(etleft):len(etleft)+len(times)], 1)

    for t in range(len(a)): # right hand side of second half of the ODE system
        # within the time range
        if len(etleft) <= t < len(etleft)+len(times):
            tt = t - len(etleft)
            for i in range(x_length):
                result[x_length+i,t] = (mat_prod[i,tt] + gamma1[i] * b_1[i,t] + flux_mu[tt,i] + flux_rec[tt,i] - delta_x[tt,i]) / gamma2[i]

        # outside the time range, no selection strength
        else:
            for i in range(x_length):
                result[x_length+i,t] = gamma1[i] * b_1[i,t] / gamma2[i]

    return result

def fun_advanced(a,b):
    """ The function that will be used if it is necessary for the BVP solver to add more nodes.
    Note that the inference may be much slower if this has to be used."""

    b_1                 = b[:x_length,:]   # the actual selection coefficients
    b_2                 = b[x_length:,:]   # the derivatives of the selection coefficients, s'
    result              = np.zeros((2*x_length,len(a))) # The RHS of the system of ODE's
    result[:x_length]   = b_2       # sets the derivatives of the selection coefficients 'b_1', equal to s'

    # create new interpolated single and double site frequencies
    single_freq_int, double_freq_int = interpolator_x(single_freq, double_freq, times, a)
    escape_freq_int                  = interpolator_ex(escape_freq, times, a)
    p_wt_int, p_mut_k_int            = interpolator_p(p_wt_freq,p_mut_k_freq,times,a,seq_length,ne)

    # use the interpolations from above to get the values of delta_x and the covariance matrix at the nodes
    flux_mu_int  = get_mutation_flux(single_freq_int, escape_freq_int, muVec) # mutation part
    flux_rec_int = get_recombination_flux(single_freq_int,p_wt_int, p_mut_k_int, trait_dis)   # recombination part
    delta_x_int  = cal_delta_x(single_freq_int,a)
    covar_int    = diffusion_matrix_at_t(single_freq_int, double_freq_int)
    covar_int    = np.swapaxes(covar_int,0,2)

    # calculate the other half of the RHS of the ODE system
    mat_prod_int  = np.sum(covar_int[:,:,:len(a)] * b_1[:,len(etleft):len(etleft)+len(times)], 1)

    for t in range(len(a)): # right hand side of second half of the ODE system
        # within the time range
        if len(etleft) <= t < len(etleft)+len(times):
            tt = t - len(etleft)
            for i in range(x_length):
                result[x_length+i,t] = (mat_prod_int[i,tt] + gamma1[i] * b_1[i,t] + flux_mu_int[tt,i] + flux_rec_int[tt,i] - delta_x_int[tt,i]) / gamma2[i]

        # outside the time range, no selection strength
        else:
            for i in range(x_length):
                result[x_length+i,t] = gamma1[i] * b_1[i,t] / gamma2[i]

    return result

# Boundary conditions
# solution to the system of differential equation with the derivative of the selection coefficients zero at the endpoints
def bc(b1,b2):
    return np.ravel(np.array([b1[x_length:],b2[x_length:]])) # s' = 0 at the extended endpoints

ss_extend = np.zeros((2*x_length,len(ExTimes)))

try:
    solution = sp.integrate.solve_bvp(fun, bc, ExTimes, ss_extend, max_nodes=10000, tol=1e-3)
except ValueError:
    print("BVP solver has to add new nodes")
    solution = sp.integrate.solve_bvp(fun_advanced, bc, ExTimes, ss_extend, max_nodes=10000, tol=1e-3)

selection_coefficients = solution.sol(ExTimes)
# removes the superfluous part of the array and only save the real time points
desired_coefficients   = selection_coefficients[:x_length,len(etleft):len(etleft)+len(times)]

end_time = time_module.time()
print(f"Execution time for CH{tag[6:]} : {end_time - start_time} seconds")


Execution time for CH058-3 : 28.50273585319519 seconds
